In [2]:
import pypdf
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from pathlib import Path
from functools import partial
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from utils_ccd import get_connection

conn = get_connection()

In [7]:
DIR_INFORMACOES = '/media/informacoes_pdf/'

def get_file_path(row):
    return Path(DIR_INFORMACOES) / row['setor'].strip() / row['arquivo']

def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as f:
            reader = pypdf.PdfReader(f)
            text = ''
            for page in reader.pages:
                text += page.extract_text() or ''
            return text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ''
    
def get_next_informacao(processo, df_last, df_informacoes):
    try:
        evento = df_last[df_last['processo'] == \
                                         processo]['evento'].iloc[0]
        return df_informacoes[(df_informacoes.evento > \
                                      evento) & \
                                        (df_informacoes['processo'] == \
                                         processo)].sort_values(by='evento').iloc[0]
    except IndexError:
        return None

def get_text_next_informacao(processo, df_last, df_informacoes):
    info = get_next_informacao(processo, df_last, df_informacoes)
    if info is not None:
        return extract_text_from_pdf(info['caminho_arquivo'])
    return ''


llm = AzureChatOpenAI(model_name="gpt-4o")

def get_descricao_objeto(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa e resume informações de processos do Tribunal de Contas do Estado do Rio Grande do Norte.

        O texto da informação é o seguinte:
        "{input}"
        Defina a Descrição resumida do objeto/assunto principal. 
        Seja sucinto e claro, não é necessário mencionar valores ou números de processos ou leis.
        Deve ser uma frase com no máximo 7 palavras, sem muitos detalhes.
                                              
        Por exemplo: 
            Cancelamento por prescrição. 
            Atualização do CADINQ. 
            Implantação de débito de desconto em folha.
            Instauração de Processo de Execução.  

        Sua resposta:
        """)

        chain = prompt | llm
        msg_obj = chain.invoke(texto)
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None
    
def get_descricao_encaminhamento(texto, proximo_texto, cip=False):
    if not proximo_texto and not cip:
        return f'Nenhum encaminhamento até o momento {datetime.datetime.now().strftime("%d/%m/%Y")}'
    try:
        if not cip:
            prompt = PromptTemplate.from_template("""
            Você é um agente que analisa encaminhamentos. A informação é sobre um 
            processo do Tribunal de Contas do Estado do Rio Grande do Norte.
            
            O texto da primeira informação é o seguinte:
            "{texto}"

            Depois veio a seguinte informação de encaminhamento:                                      
            "{proximo_texto}"
                                                
            Seja sucinto e claro, não é necessário mencionar valores ou números de processos ou leis.
            Deve ser uma frase ou duas, sem muitos detalhes.
                                                
            Por exemplo: 
                Encaminhamento do Ministério Público à Procuradoria Geral do Estado.
                
                
                                                
            Sua resposta:
            """)
        else:
            prompt = PromptTemplate.from_template("""
            Você é um agente que analisa encaminhamentos. A informação é sobre um 
            processo do Tribunal de Contas do Estado do Rio Grande do Norte.
            
            O texto da informação é o seguinte:
            "{texto}"

            Identifique na informação o encaminhamento do processo.                        
            Seja sucinto e claro, não é necessário mencionar valores ou números de processos ou leis.
            Deve ser uma frase ou duas, sem muitos detalhes.
                                                
            Por exemplo: 
                Encaminhamento do Ministério Público à Procuradoria Geral do Estado.
                                                
            Sua resposta:
            """)

        chain = prompt | llm
        msg_obj = chain.invoke({"texto":texto, "proximo_texto":proximo_texto})
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

def generate_excel(setor, ano, meses, filename):
    sql_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE setor = '{setor}'
    and year(inf.data_resumo) = {ano}
    and month(inf.data_resumo) IN ({', '.join(map(str, meses))})
    '''
    informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
    informacoes_processos['caminho_arquivo'] = informacoes_processos.apply(get_file_path, axis=1)
    
    informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Pagamento da parcela')]
    informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Capa')]
    informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Evento do Processo Original')]

    informacoes_setor = informacoes_processos[informacoes_processos['setor'].str.strip() == setor]
    informacoes_setor_last = informacoes_setor.sort_values(by='evento', ascending=False).drop_duplicates(subset='processo', keep='first')

    informacoes_setor_last['resumo'].fillna('', inplace=True)
    
    informacoes_setor_last = informacoes_setor_last[~informacoes_setor_last['resumo'].str.contains('Pagamento da parcela')]

    informacoes_setor_last['texto'] = informacoes_setor_last['caminho_arquivo'].apply(extract_text_from_pdf)

    processos_busca = ', '.join(f"'{proc}'" for proc in informacoes_setor_last['processo'].unique())

    sql_all_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos_busca})
    '''
    all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)
    all_informacoes_processos['caminho_arquivo'] = all_informacoes_processos.apply(get_file_path, axis=1)

    get_text_next_informacao_partial = partial(get_text_next_informacao, \
                                               df_last=informacoes_setor_last, \
                                                df_informacoes=all_informacoes_processos)
    informacoes_setor_last['proximo_texto'] = informacoes_setor_last['processo'].apply(get_text_next_informacao_partial)

    informacoes_setor_last['descricao_objeto'] = informacoes_setor_last['texto'].apply(get_descricao_objeto)
    
    informacoes_setor_last['descricao_encaminhamento'] = informacoes_setor_last.apply(
        lambda row: get_descricao_encaminhamento(row['texto'], row['proximo_texto'], setor == 'CIP'), axis=1
    )
    
    sql_processos = f'''
    SELECT CONCAT(p.numero_processo, '/', p.ano_processo) AS processo,
           	COALESCE(vgo.nomeorgao COLLATE SQL_Latin1_General_CP1_CI_AI, p.interessado) AS jurisdicionado,
            rel.nome as relator,
            p.assunto,
            (SELECT TOP 1 il.recebido_em
	            FROM processo.dbo.Lotes AS l
	            INNER JOIN processo.dbo.Itens_Lote AS il 
	            	ON il.IdLote = l.IdLote
	            INNER JOIN processo.dbo.Processos AS p2 
	            	ON p2.numero_processo = il.numero_processo AND p2.ano_processo = il.ano_processo
	            WHERE p2.numero_processo = p.numero_processo AND p2.ano_processo = p.ano_processo
	            AND destino = '{setor}'
	            ORDER BY l.enviado_em DESC
            ) as data_recebimento,
            (SELECT TOP 1 l.enviado_em
	            FROM processo.dbo.Lotes AS l
	            INNER JOIN processo.dbo.Itens_Lote AS il 
	            	ON il.IdLote = l.IdLote
	            INNER JOIN processo.dbo.Processos AS p2 
	            	ON p2.numero_processo = il.numero_processo AND p2.ano_processo = il.ano_processo
	            WHERE p2.numero_processo = p.numero_processo AND p2.ano_processo = p.ano_processo
	            AND origem = '{setor}'
	            ORDER BY l.enviado_em DESC
            ) as data_saida
     
	FROM processo.dbo.Processos p
	LEFT JOIN processo.dbo.Relator AS rel ON p.codigo_relator = rel.codigo
    LEFT JOIN Bdc.dbo.vw_Gen_Orgao vgo ON vgo.IdOrgao = p.IdOrgaoEnvolvido
	WHERE CONCAT(p.numero_processo, '/', p.ano_processo) in ({processos_busca})
        '''
    df_processos = pd.read_sql(sql_processos, conn)

    processos_informacoes_setor = informacoes_setor_last.merge(df_processos, on='processo', how='left')
    
    tab_setor = processos_informacoes_setor[['processo', 'jurisdicionado', 'relator', 
                            'data_recebimento', 'data_saida', 
                            'descricao_objeto', 'descricao_encaminhamento']]
    tab_setor.fillna('', inplace=True)
    tab_setor['data_recebimento'] = pd.to_datetime(tab_setor['data_recebimento'], errors='coerce')
    tab_setor['data_recebimento']= tab_setor['data_recebimento'].dt.strftime('%d/%m/%Y')
    tab_setor['data_saida'] = pd.to_datetime(tab_setor['data_saida'], errors='coerce')
    tab_setor['data_saida'] = tab_setor['data_saida'].dt.strftime('%d/%m/%Y')
    tab_setor.sort_values(by='data_saida', ascending=True, inplace=True)
    tab_setor.to_excel(filename, index=False)
    

In [8]:
generate_excel('CIP', 2025, [4,5], 'saidas/sisdgf/processos_cip.xlsx')

/tmp/ipykernel_13520/2345049978.py:127: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
/tmp/ipykernel_13520/2345049978.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)
/tmp/ipykernel_13520/2345049978.py:199: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_processos = pd.read_sql(sql_processos, conn)
/tmp/ipykernel_13520/2345049978.py:206: SettingWithCopyWarning: 
A value is t

In [ ]:
generate_excel('CCD', 2025, [4,5], 'saidas/sisdgf/processos_ccd.xlsx')

/tmp/ipykernel_13520/2345049978.py:127: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
/tmp/ipykernel_13520/2345049978.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


Error processing  
 
 
Diretoria de Instrução Proce ssual e Controle de Decisõe s 
Av. Pres. GETÚLIO VARGAS, 690 – Ed. Dr. MÚCIO VILAR RIBEIRO DANTAS 
DAE – 1º andar-Fone (0xx84) 3642-7346 / 3642-7350 
PETRÓPOLIS – NATAL / R N – CEP: 59012-360 
 
 
 
Processo nº: 003275/2023 
Assunto: Execução da decisão proferida nos autos do processo 200131/2021 
Responsável:  Jorge Vinicius de Oliveira Fernandes - Prefeito - CPF:01673675441 
                                                     
C E R T I D Ã O 
 
 
Diante da de terminação da Relatoria para reiterar  a notificação à Prefeitura 
Municipal de Antônio Martins , i nformo que o Sr.  Jorge Vinicius de Oliveira Fernandes  não 
possui vínculo com a administração pública da presente data, conforme pesquisa no SIAI-DP, 
no Portal d a transparência do Governo Federal, Portal da transparência da câmara dos 
deputados e do Senado Federal.  Complemento a informação reforçando que o Sr. Jorge 
Vinicius de Oliveira Fernandes era Prefeito do Municípi